# Prompts

Prompts are essentially the instructions or questions you give to AI models to get them to generate specific responses.\
Think of them as the conversation starter that guides what the AI will say back to you.

This has even given rise to a field called "Prompt Engineering" — a discipline focused on developing and optimizing prompts for effective use of language models.

In this notebook, we'll explore prompts in Spring AI:
* Basic prompts
* Message types
* Prompting techniques

Let's add the necessary dependencies:

In [1]:
%useLatestDescriptors
%use spring-ai-anthropic

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY") ?: "YOUR_ANTHROPIC_API_KEY"

Just like in the **`Intro`** notebook, let's create `ChatOptions` and a `ChatModel`.

In [3]:
val anthropicApi = AnthropicApi.builder().apiKey(apiKey).build()
val anthropicOptions = AnthropicChatOptions.builder()
    .model(AnthropicApi.ChatModel.CLAUDE_SONNET_4_0)
    .temperature(0.7)
    .maxTokens(1024)
    .build()

val chatCompletion = AnthropicChatModel.builder()
    .anthropicApi(anthropicApi)
    .defaultOptions(anthropicOptions)
    .build()

## What makes a prompt?

A prompt is simply a text request: "tell me a joke" or "write a poem about mountains"

Let's ask our LLM to generate a haiku:

In [4]:
chatCompletion.call("Generate a hokku")

Here's a hokku for you:

Morning frost melts—
a single drop catches light
on the bamboo leaf

If we're using `ChatClient` and the `Prompt` class, the request would look like this:

In [5]:
val chatClient = ChatClient.create(chatCompletion)

val prompt = Prompt("Generate a hokku")
chatClient.prompt(prompt).call().content()

Here's a hokku for you:

Morning dew clings tight—
even the spider's silk holds
autumn's first farewell

## Type of messages

In AI interactions, there are several message types (roles):
* User — message from the user
* Assistant — message from the AI
* System — instructions that guide the AI's behavior
* Tool — used for function calling

### User messages

We've been writing user messages all along.

Let's explicitly define them now:

In [6]:
val messages = Prompt(UserMessage("Generate a hokku"), UserMessage("what's name of this hokku?"))
chatClient.prompt(messages).call().content()

Here's a hokku I've created:

**Morning dew glistens**
**on the spider's silver web—**
**autumn light breaking**

Regarding the name: Traditional hokku (and haiku) typically don't have titles. They are meant to stand alone as complete, self-contained moments of observation. The poem itself is simply called a "hokku" - that's both the form and the identifier. If you needed to reference this particular one, you might refer to it by its first line: "Morning dew glistens."

### System messages

A system message tells the LLM how it should behave.
In Spring-AI, you can send a system message in several ways.

For instance, you can use a special function for `ChatClient` or create an instance of a system message and pass it to the LLM directly.

In [7]:
chatClient
    .prompt()
    .system("You are a financial expert. Answer briefly.")
    .user("If I had a time machine, should I buy Bitcoin in 2011?")
    .call()
    .content()

Yes, absolutely. Bitcoin was trading for around $1-$30 in 2011 and peaked near $69,000 in 2021 - that's roughly a 2,000-69,000x return depending on when in 2011 you bought.

However, keep in mind:
- You'd need diamond hands to hold through multiple 80%+ crashes
- Early exchanges were risky (Mt. Gox collapsed in 2014)
- You'd need secure storage methods that didn't exist yet
- The regulatory landscape was completely unknown

But yes, even buying a small amount and holding would have been life-changing wealth creation.

In [ ]:
val messages = listOf(
    SystemMessage("You are a financial expert. Answer briefly."),
    UserMessage("If I had a time machine, should I buy Bitcoin in 2011?")
)

chatClient.prompt(Prompt(messages)).call().content()

### Assistant messages

As mentioned earlier, an assistant message is essentially a message from the LLM.

Let's create a simple conversation example:
we'll give the LLM a system instruction,
send a request,
and after receiving a response,
ask for clarification on a specific point.

In [ ]:
val messages = mutableListOf(
    SystemMessage("You are an assistant who always answers very briefly, using no more than 10 words."),
    UserMessage("Tell me about the capital of France and its landmarks")
)
val assistantMessage = chatClient.prompt(Prompt(messages.toList())).call().chatResponse()!!.result.output
assistantMessage

In [ ]:
messages.add(assistantMessage)
messages.add(UserMessage("Tell me about the third landmark"))

chatClient.prompt(Prompt(messages.toList())).call().content()

We've essentially created a simple dialogue between human and machine.

Notice that in the last message,
we deliberately formulated the request as:
`"Tell me about the third landmark"`.
If we hadn't sent this request along with the LLM's response about French landmarks,
we wouldn't have received a meaningful answer.

## Prompt templates

Spring AI provides PromptTemplate for working with prompts. This class uses the OSS
[String Template](https://www.stringtemplate.org/)
engine developed by Terence Parr for constructing and managing prompts.

This class allows you to use resources for prompts, making them easier to manage and localize in different languages.
It also lets you insert your data into the prompt,
which is especially useful when developing RAG applications (which we'll learn more about in future notebooks).

Let's write a simple example using PromptTemplate:

In [ ]:
fun capital(): PromptTemplate {
    val message = "The two largest cities in {country}"
    return PromptTemplate(message)
}

val prompt = capital().create(mapOf("country" to "France"))
chatClient.prompt(prompt).call().content()

## Prompting techniques

There are various techniques for crafting effective prompts that help us get better results from language models.
These techniques range from simple to complex and can dramatically improve the quality of AI responses.

We will consider techniques such as:
* Zero-Shot Prompting
* Few-Shot Prompting
* Chain-of-Thought (CoT) Prompting
* Meta Prompting
* Generate Knowledge Prompting
* Prompt Chaining

These are far from all the techniques. There are many more.

### Zero-Shot Prompting

Zero-shot prompting is a technique where the model performs a task based on direct instruction without being provided examples or demonstrations.
The model relies solely on its pre-training knowledge.

In [ ]:
chatClient
    .prompt("""
    Classify the text as neutral, negative, or positive.
    Text: In my opinion, this restaurant is quite ordinary.
    Tonality:
    """)
    .call()
    .content()

### Few-Shot Prompting

Few-shot prompting is a technique where several examples (demonstrations) of task performance are included in the prompt to help the model understand exactly how to perform a similar task.
Instead of training the model from scratch, we provide context through examples directly in the prompt.

In [ ]:
chatClient
    .prompt("""
    "Zumbrik" is a small fluffy animal inhabiting the Altai Mountains. Example sentence with the word zumbrik:
    During our expedition to the Altai Mountains we met a family of cute zumbriks.

    "Fyrkotat" means to quickly rotate in one place. Example sentence with the word fyrkotat:
    """)
    .call()
    .content()

In this example, we provided the model with one example (1-shot) of using a made-up word in a sentence.
Based on this,
the model understood the task and was able to create a similar sentence with another made-up word,
following the demonstrated pattern.

### Chain-of-Thought (CoT) Prompting

Chain-of-Thought is a prompting technique that encourages the model to show intermediate steps of reasoning before providing the final answer.
This is especially useful for complex tasks requiring mathematical calculations,
logical analysis, or multi-step reasoning.

In [ ]:
chatClient
    .prompt("""
    In the group of numbers 15, 8, 3, 22, 7, 14, 26, do the odd numbers sum to an even number?
    Let's reason step by step.
    """)
    .call()
    .content()

### Meta Prompting

Meta prompting is an advanced technique that focuses on the structural and syntactic aspects of tasks rather than specific content details.
It creates an abstract, structured way of interacting with the LLM,
where the form and pattern of information are more important than the content itself.

In [ ]:
chatClient
    .prompt("""
    In problems of the format [problem P → solution S], follow this structure:
    1. Define the variables from P
    2. Construct an equation based on P
    3. Solve the equation to find S
    4. Verify the solution by substitution

    Problem: A store had x apples. After selling 15 apples and then another 1/3 of the remaining apples, the store had 20 apples left. How many apples were there initially?
    """)
    .call()
    .content()

In this example, the meta prompt sets a general structure for approaching problem-solving,
not focusing on specific content but offering a universal template for analysis and solution.
The model follows this structure, applying it to the specific problem.

### Generate Knowledge Prompting

Generate Knowledge Prompting is a technique where the model first generates factual knowledge about a topic and then uses this knowledge to form a more accurate and well-founded answer.
This technique is especially useful for tasks requiring common sense or factual accuracy.

In [ ]:
val knowledge = chatClient
    .prompt("""
    Request: Are lichens harmful to trees?
    Generate knowledge:
    """)
    .call()
    .content()

knowledge

In [ ]:
chatClient
    .prompt("""
    Answer briefly: Are lichens harmful to trees?

    knowledge: $knowledge
    """)
    .call()
    .content()

This technique allows the model to first gather relevant knowledge and then use it to form a more accurate and informative response.

### Prompt Chaining

Prompt Chaining is a technique where a complex task is broken down into several sequential subtasks.
The answer from one prompt becomes the input data for the next, creating a chain of operations.
This allows solving complex tasks, increases transparency, controllability, and reliability when working with LLMs.

In [ ]:
val ctryCap = chatClient
    .prompt("""
    Extract all mentions of countries and their capitals from the following text.  The answer should be a list in the format "Country: Capital".

    Text:
    France is famous for the Eiffel Tower in Paris, and Germany is known for its automotive industry with headquarters in Berlin.  Meanwhile, tourists enjoy the picturesque views of Rome in Italy and the castles near Madrid in Spain.
    """)
    .call()
    .content()

In [ ]:
chatClient
    .prompt("""
    Based on the following list of countries and their capitals, create a short guide to the three most interesting sights in each capital city:

    $ctryCap
    """)
    .call()
    .content()

Working with prompts is fundamental for both AI application users and developers.
Spring AI and Kotlin provide a convenient and powerful API for this purpose.

Check out the next notebook to learn more about Kotlin and Spring AI!